# **Revisión de resultados**

Este notebbok detecta automáticamente el último *_svm_preds.csv de inference/ y abre una interfaz Gradio para revisar los resultados del pipeline BO_SVM.

Finalmente guarda los datos para un posterior re entrenamiento del modelo.

La interfaz muestra: origen_pdf, page, origen_flags, score_svm, pred_pertinente.

Funcionalidades:

1. Editar texto del fragmento.
2. ✅ Confirmar lo que dice el modelo.
3. 👍 Corregir a “pertinente”.
4. 👎 Corregir a “no pertinente”.
5. ⏭ Pasar al siguiente sin revisar.
6. El CSV original se va actualizando (columna label y contexto si se edita).
7. “Exportar revisados a master” (consolidar para reentrenar). Se genera/actualiza train_feedback_master.csv con todos los registros revisados.

Este train_feedback_master.csv se agrega al train_master cuando se planifique el reentrenamiento del SVM.

## **Celda 1 – Instalar Gradio, montar Drive y definir rutas**

In [1]:
!pip install gradio > /dev/null

from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import gradio as gr

BASE = "/content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml"

# Carpeta donde el notebook de inferencia guarda los *_svm_preds.csv
DIR_OUT = os.path.join(BASE, "inference")

# Archivo maestro donde vamos a ir acumulando feedback para reentrenar
FEEDBACK_MASTER = os.path.join(BASE, "data", "labels", "train_feedback_master.csv")

os.makedirs(os.path.dirname(FEEDBACK_MASTER), exist_ok=True)
print("BASE:", BASE)
print("DIR_OUT:", DIR_OUT)
print("FEEDBACK_MASTER:", FEEDBACK_MASTER)

Mounted at /content/drive
BASE: /content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml
DIR_OUT: /content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/inference
FEEDBACK_MASTER: /content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/data/labels/train_feedback_master.csv


## **Celda 2 – Cargar el CSV de predicciones más reciente**

Esta celda:

1. Busca el último *_svm_preds.csv en DIR_OUT.

2. Lo carga.

3. Si no tiene columna label, la crea con -1 = sin revisar.

In [2]:
def get_latest_preds_csv(dir_path=DIR_OUT):
    csvs = [
        os.path.join(dir_path, f)
        for f in os.listdir(dir_path)
        if f.lower().endswith("_svm_preds.csv")
    ]
    if not csvs:
        raise FileNotFoundError(f"No hay CSV *_svm_preds.csv en {dir_path}")
    latest = max(csvs, key=os.path.getmtime)
    return latest

def load_review_df(path):
    # mismos parámetros que usaste en inferencia
    df = pd.read_csv(path, sep=";", encoding="utf-8-sig")
    if "label" not in df.columns:
        df["label"] = -1  # -1 = sin revisar, 0 = no pertinente, 1 = pertinente
    return df

CSV_PATH = get_latest_preds_csv(DIR_OUT)
print("Revisando CSV:", CSV_PATH)

df_init = load_review_df(CSV_PATH)
print("Filas totales:", len(df_init))
print("Sin revisar:", (df_init["label"] == -1).sum())

Revisando CSV: /content/drive/MyDrive/IA/Proyectos/Análisis Boletín Oficial/boletin-ml/inference/20251104_svm_preds.csv
Filas totales: 53
Sin revisar: 53


## **Celda 3 – Helpers para mostrar meta, progreso y navegación**

Esta celda define:

- cómo se muestra la información del registro actual,

- cómo se calcula el progreso,

- cómo moverse al siguiente sin revisar.

In [3]:
def format_meta(row, idx, total):
    return (
        f"**Registro {idx+1} de {total}**  \n"
        f"**Boletín:** `{row.get('origen_pdf', '')}`  \n"
        f"**Página:** {row.get('page', 'N/D')}  \n"
        f"**Flags:** {row.get('origen_flags', '')}  \n"
        f"**Score SVM:** {row.get('score_svm', 0):.3f}  \n"
        f"**Predicción modelo (pred_pertinente):** {int(row.get('pred_pertinente', 0))}"
    )

def format_progress(df, idx):
    total = len(df)
    done = int((df["label"] != -1).sum())
    return (
        f"Revisados: {done}/{total} registros  \n"
        f"Posición actual: {idx+1}/{total}"
    )

def find_next_unlabeled(df, start_idx):
    """
    Busca el próximo índice con label == -1 a partir de start_idx+1.
    Si no queda ninguno, se queda en el actual.
    """
    n = len(df)
    idx = start_idx + 1
    while idx < n and df.at[idx, "label"] != -1:
        idx += 1
    if idx >= n:
        idx = start_idx
    return idx

def get_view(df, idx):
    row = df.iloc[idx]
    meta = format_meta(row, idx, len(df))
    progreso = format_progress(df, idx)
    contexto = str(row.get("contexto", ""))
    return meta, contexto, progreso

## **Celda 4 – Funciones de acción (confirmar / corregir / exportar)**

Estas funciones:

- aplican la etiqueta (0/1),
- actualizan el contexto si lo editaste,
- guardan de vuelta el CSV original,
- avanzan al siguiente sin revisar,
- exportan a un “master” acumulado.

In [4]:
def save_df(df):
    df.to_csv(CSV_PATH, index=False, sep=";", encoding="utf-8-sig")

def apply_label(df, idx, new_label, new_context):
    df = df.copy()
    df.at[idx, "label"] = int(new_label)
    df.at[idx, "contexto"] = new_context
    save_df(df)

    next_idx = find_next_unlabeled(df, idx)
    meta, contexto, progreso = get_view(df, next_idx)
    return df, next_idx, meta, contexto, progreso

def confirm_pred(df, idx, contexto):
    """
    Confirmar la predicción del modelo:
      label = pred_pertinente
    """
    pred = int(df.at[idx, "pred_pertinente"])
    return apply_label(df, idx, pred, contexto)

def mark_pos(df, idx, contexto):
    """Marcar explícitamente como pertinente (1)."""
    return apply_label(df, idx, 1, contexto)

def mark_neg(df, idx, contexto):
    """Marcar explícitamente como no pertinente (0)."""
    return apply_label(df, idx, 0, contexto)

def skip_row(df, idx, contexto):
    """No cambia label ni texto; salta al próximo sin revisar."""
    df = df.copy()
    next_idx = find_next_unlabeled(df, idx)
    meta, contexto2, progreso = get_view(df, next_idx)
    return df, next_idx, meta, contexto2, progreso

def export_to_master(df):
    """
    Exporta todos los registros con label 0/1 a un CSV acumulado
    (para luego sumarlos al train_master).
    """
    df_valid = df[df["label"].isin([0, 1])].copy()

    # columnas más útiles para reentrenar
    cols = ["contexto", "label", "origen_pdf", "page", "origen_flags", "score_svm", "pred_pertinente"]
    for c in cols:
        if c not in df_valid.columns:
            df_valid[c] = None

    df_valid = df_valid[cols]

    if os.path.exists(FEEDBACK_MASTER):
        master = pd.read_csv(FEEDBACK_MASTER, sep=";", encoding="utf-8-sig")
        combined = pd.concat([master, df_valid], ignore_index=True)
        combined = combined.drop_duplicates(subset=["origen_pdf", "page", "contexto"])
    else:
        combined = df_valid.drop_duplicates(subset=["origen_pdf", "page", "contexto"])

    combined.to_csv(FEEDBACK_MASTER, index=False, sep=";", encoding="utf-8-sig")
    return f"Exportados {len(df_valid)} registros de este CSV. Total acumulado en master: {len(combined)}"

## **Celda 5 – Lanzar la interfaz Gradio**

Interfaz simple:

- Muestra meta (archivo, página, flags, score, predicción).
- Cuadro de texto editable con el fragmento.

Botones:

✅ Confirmar predicción

👍 Marcar como pertinente (1)

👎 Marcar como no pertinente (0)

⏭ Omitir / siguiente

💾 Exportar revisados a master

In [5]:
# Estados iniciales
meta0, contexto0, progreso0 = get_view(df_init, 0)

with gr.Blocks() as demo:
    gr.Markdown("### Revisión de predicciones SVM sobre Boletín Oficial")

    df_state = gr.State(df_init)
    idx_state = gr.State(0)

    meta = gr.Markdown(meta0)
    contexto = gr.Textbox(value=contexto0, lines=10, label="Fragmento (editable)")
    progreso = gr.Markdown(progreso0)

    with gr.Row():
        btn_confirm = gr.Button("✅ Confirmar predicción")
        btn_pos = gr.Button("👍 Marcar como pertinente (1)")
        btn_neg = gr.Button("👎 Marcar como no pertinente (0)")
        btn_skip = gr.Button("⏭ Omitir / siguiente")

    msg_export = gr.Markdown("")
    btn_export = gr.Button("💾 Exportar revisados a master")

    btn_confirm.click(
        confirm_pred,
        inputs=[df_state, idx_state, contexto],
        outputs=[df_state, idx_state, meta, contexto, progreso],
    )

    btn_pos.click(
        mark_pos,
        inputs=[df_state, idx_state, contexto],
        outputs=[df_state, idx_state, meta, contexto, progreso],
    )

    btn_neg.click(
        mark_neg,
        inputs=[df_state, idx_state, contexto],
        outputs=[df_state, idx_state, meta, contexto, progreso],
    )

    btn_skip.click(
        skip_row,
        inputs=[df_state, idx_state, contexto],
        outputs=[df_state, idx_state, meta, contexto, progreso],
    )

    btn_export.click(
        export_to_master,
        inputs=[df_state],
        outputs=[msg_export],
    )

demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>